# Healthcare AI Implementation Guide - Chapter 2
## Mathematical Foundations for Healthcare AI

This notebook provides comprehensive implementations of the mathematical concepts essential for healthcare AI applications.

**Learning Objectives:**
- Understand probability theory applications in clinical decision making
- Implement Bayesian inference for diagnostic reasoning
- Apply information theory to healthcare data analysis
- Master optimization techniques for clinical machine learning
- Implement causal inference methods for healthcare research

**Prerequisites:**
- Basic knowledge of calculus and linear algebra
- Python programming experience
- Understanding of clinical concepts from Chapter 1

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Healthcare AI Mathematical Foundations - Interactive Tutorial")
print("=" * 60)

## Section 1: Probability Theory in Clinical Decision Making

Probability theory forms the foundation of evidence-based medicine and clinical decision support systems. We'll explore how to apply probabilistic reasoning to diagnostic and therapeutic decisions.

In [ ]:
class ClinicalProbabilityCalculator:
    """
    Comprehensive probability calculator for clinical applications.
    
    This class implements various probability calculations commonly used
    in clinical decision making, including diagnostic test interpretation
    and risk assessment.
    """
    
    def __init__(self):
        """Initialize the clinical probability calculator."""
        self.test_results = {}
        
    def calculate_diagnostic_accuracy(self, sensitivity, specificity, prevalence):
        """
        Calculate diagnostic test accuracy metrics.
        
        Parameters:
        -----------
        sensitivity : float
            Test sensitivity (true positive rate)
        specificity : float
            Test specificity (true negative rate)
        prevalence : float
            Disease prevalence in the population
            
        Returns:
        --------
        dict : Dictionary containing all diagnostic metrics
        """
        # Calculate positive and negative predictive values
        ppv = (sensitivity * prevalence) / (
            sensitivity * prevalence + (1 - specificity) * (1 - prevalence)
        )
        
        npv = (specificity * (1 - prevalence)) / (
            specificity * (1 - prevalence) + (1 - sensitivity) * prevalence
        )
        
        # Calculate likelihood ratios
        lr_positive = sensitivity / (1 - specificity)
        lr_negative = (1 - sensitivity) / specificity
        
        # Calculate accuracy
        accuracy = sensitivity * prevalence + specificity * (1 - prevalence)
        
        return {
            'sensitivity': sensitivity,
            'specificity': specificity,
            'prevalence': prevalence,
            'ppv': ppv,
            'npv': npv,
            'lr_positive': lr_positive,
            'lr_negative': lr_negative,
            'accuracy': accuracy
        }
    
    def bayesian_diagnostic_update(self, prior_probability, likelihood_ratio):
        """
        Update diagnostic probability using Bayes' theorem.
        
        Parameters:
        -----------
        prior_probability : float
            Prior probability of disease (pre-test probability)
        likelihood_ratio : float
            Likelihood ratio of the test result
            
        Returns:
        --------
        float : Posterior probability (post-test probability)
        """
        # Convert probability to odds
        prior_odds = prior_probability / (1 - prior_probability)
        
        # Apply likelihood ratio
        posterior_odds = prior_odds * likelihood_ratio
        
        # Convert back to probability
        posterior_probability = posterior_odds / (1 + posterior_odds)
        
        return posterior_probability
    
    def plot_diagnostic_performance(self, sensitivity_range, specificity, prevalence):
        """
        Plot how diagnostic performance varies with sensitivity.
        """
        sensitivities = np.linspace(sensitivity_range[0], sensitivity_range[1], 100)
        ppvs = []
        npvs = []
        
        for sens in sensitivities:
            metrics = self.calculate_diagnostic_accuracy(sens, specificity, prevalence)
            ppvs.append(metrics['ppv'])
            npvs.append(metrics['npv'])
        
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(sensitivities, ppvs, 'b-', linewidth=2, label='PPV')
        plt.plot(sensitivities, npvs, 'r-', linewidth=2, label='NPV')
        plt.xlabel('Sensitivity')
        plt.ylabel('Predictive Value')
        plt.title(f'Predictive Values vs Sensitivity\n(Specificity={specificity:.2f}, Prevalence={prevalence:.2f})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # ROC-like curve
        plt.subplot(1, 2, 2)
        fpr = 1 - specificity
        plt.plot(sensitivities, [fpr] * len(sensitivities), 'g--', alpha=0.7, label='False Positive Rate')
        plt.plot(sensitivities, sensitivities, 'b-', linewidth=2, label='True Positive Rate')
        plt.xlabel('Sensitivity (True Positive Rate)')
        plt.ylabel('Rate')
        plt.title('Sensitivity vs False Positive Rate')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Example: Diagnostic test evaluation
calc = ClinicalProbabilityCalculator()

# Example: COVID-19 rapid test
covid_metrics = calc.calculate_diagnostic_accuracy(
    sensitivity=0.85,  # 85% sensitivity
    specificity=0.95,  # 95% specificity
    prevalence=0.05    # 5% prevalence
)

print("COVID-19 Rapid Test Performance:")
print(f"Sensitivity: {covid_metrics['sensitivity']:.1%}")
print(f"Specificity: {covid_metrics['specificity']:.1%}")
print(f"Positive Predictive Value: {covid_metrics['ppv']:.1%}")
print(f"Negative Predictive Value: {covid_metrics['npv']:.1%}")
print(f"Positive Likelihood Ratio: {covid_metrics['lr_positive']:.1f}")
print(f"Negative Likelihood Ratio: {covid_metrics['lr_negative']:.2f}")

# Visualize performance
calc.plot_diagnostic_performance(
    sensitivity_range=(0.5, 1.0),
    specificity=0.95,
    prevalence=0.05
)

## Section 2: Bayesian Inference for Clinical Decision Making

Bayesian inference provides a principled framework for updating beliefs based on new evidence. This is particularly valuable in clinical settings where we need to combine prior knowledge with new test results.

In [ ]:
class BayesianClinicalInference:
    """
    Bayesian inference framework for clinical applications.
    
    This class implements Bayesian methods for clinical decision making,
    including diagnostic reasoning and treatment effect estimation.
    """
    
    def __init__(self):
        """Initialize Bayesian inference system."""
        self.prior_distributions = {}
        self.posterior_distributions = {}
        
    def beta_binomial_conjugate(self, prior_alpha, prior_beta, successes, trials):
        """
        Perform Beta-Binomial conjugate analysis.
        
        This is commonly used for analyzing treatment success rates,
        diagnostic test performance, and other binary outcomes.
        
        Parameters:
        -----------
        prior_alpha, prior_beta : float
            Parameters of the Beta prior distribution
        successes : int
            Number of observed successes
        trials : int
            Total number of trials
            
        Returns:
        --------
        dict : Posterior distribution parameters and statistics
        """
        # Update parameters using conjugate prior
        posterior_alpha = prior_alpha + successes
        posterior_beta = prior_beta + trials - successes
        
        # Calculate statistics
        prior_mean = prior_alpha / (prior_alpha + prior_beta)
        posterior_mean = posterior_alpha / (posterior_alpha + posterior_beta)
        
        # Credible intervals
        posterior_ci_lower = stats.beta.ppf(0.025, posterior_alpha, posterior_beta)
        posterior_ci_upper = stats.beta.ppf(0.975, posterior_alpha, posterior_beta)
        
        return {
            'prior_alpha': prior_alpha,
            'prior_beta': prior_beta,
            'posterior_alpha': posterior_alpha,
            'posterior_beta': posterior_beta,
            'prior_mean': prior_mean,
            'posterior_mean': posterior_mean,
            'credible_interval': (posterior_ci_lower, posterior_ci_upper),
            'observed_rate': successes / trials if trials > 0 else 0
        }
    
    def normal_normal_conjugate(self, prior_mean, prior_var, data):
        """
        Perform Normal-Normal conjugate analysis.
        
        Used for continuous outcomes like blood pressure, lab values, etc.
        
        Parameters:
        -----------
        prior_mean : float
            Prior mean
        prior_var : float
            Prior variance
        data : array-like
            Observed data points
            
        Returns:
        --------
        dict : Posterior distribution parameters and statistics
        """
        data = np.array(data)
        n = len(data)
        sample_mean = np.mean(data)
        
        # Assume known variance for simplicity (can be extended)
        data_var = np.var(data, ddof=1) if n > 1 else 1.0
        
        # Update parameters
        posterior_var = 1 / (1/prior_var + n/data_var)
        posterior_mean = posterior_var * (prior_mean/prior_var + n*sample_mean/data_var)
        
        # Credible intervals
        posterior_std = np.sqrt(posterior_var)
        ci_lower = stats.norm.ppf(0.025, posterior_mean, posterior_std)
        ci_upper = stats.norm.ppf(0.975, posterior_mean, posterior_std)
        
        return {
            'prior_mean': prior_mean,
            'prior_var': prior_var,
            'posterior_mean': posterior_mean,
            'posterior_var': posterior_var,
            'sample_mean': sample_mean,
            'sample_size': n,
            'credible_interval': (ci_lower, ci_upper)
        }
    
    def plot_bayesian_update(self, prior_params, posterior_params, distribution_type='beta'):
        """
        Visualize Bayesian updating process.
        """
        plt.figure(figsize=(12, 5))
        
        if distribution_type == 'beta':
            x = np.linspace(0, 1, 1000)
            
            # Prior distribution
            prior_pdf = stats.beta.pdf(x, prior_params['prior_alpha'], prior_params['prior_beta'])
            
            # Posterior distribution
            posterior_pdf = stats.beta.pdf(x, posterior_params['posterior_alpha'], posterior_params['posterior_beta'])
            
            plt.subplot(1, 2, 1)
            plt.plot(x, prior_pdf, 'b-', linewidth=2, label=f'Prior: Beta({prior_params["prior_alpha"]}, {prior_params["prior_beta"]})')
            plt.plot(x, posterior_pdf, 'r-', linewidth=2, label=f'Posterior: Beta({posterior_params["posterior_alpha"]}, {posterior_params["posterior_beta"]})')
            plt.axvline(posterior_params['observed_rate'], color='green', linestyle='--', label='Observed Rate')
            plt.xlabel('Success Rate')
            plt.ylabel('Density')
            plt.title('Bayesian Update: Prior vs Posterior')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Credible interval visualization
            plt.subplot(1, 2, 2)
            ci_lower, ci_upper = posterior_params['credible_interval']
            x_ci = x[(x >= ci_lower) & (x <= ci_upper)]
            y_ci = stats.beta.pdf(x_ci, posterior_params['posterior_alpha'], posterior_params['posterior_beta'])
            
            plt.plot(x, posterior_pdf, 'r-', linewidth=2, label='Posterior Distribution')
            plt.fill_between(x_ci, y_ci, alpha=0.3, color='red', label='95% Credible Interval')
            plt.axvline(posterior_params['posterior_mean'], color='red', linestyle='-', label='Posterior Mean')
            plt.xlabel('Success Rate')
            plt.ylabel('Density')
            plt.title(f'95% Credible Interval: [{ci_lower:.3f}, {ci_upper:.3f}]')
            plt.legend()
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Example: Treatment efficacy analysis
bayesian_system = BayesianClinicalInference()

# Example: New drug trial
# Prior: Skeptical prior based on historical data
prior_alpha = 2  # Equivalent to 2 prior successes
prior_beta = 8   # Equivalent to 8 prior failures (20% success rate)

# Observed data: 15 successes out of 20 patients
successes = 15
trials = 20

# Perform Bayesian analysis
drug_analysis = bayesian_system.beta_binomial_conjugate(
    prior_alpha, prior_beta, successes, trials
)

print("Drug Efficacy Bayesian Analysis:")
print(f"Prior belief (success rate): {drug_analysis['prior_mean']:.1%}")
print(f"Observed success rate: {drug_analysis['observed_rate']:.1%}")
print(f"Updated belief (posterior mean): {drug_analysis['posterior_mean']:.1%}")
print(f"95% Credible Interval: [{drug_analysis['credible_interval'][0]:.1%}, {drug_analysis['credible_interval'][1]:.1%}]")

# Visualize the Bayesian update
bayesian_system.plot_bayesian_update(drug_analysis, drug_analysis, 'beta')

## Section 3: Information Theory in Healthcare Data Analysis

Information theory provides powerful tools for analyzing healthcare data, including feature selection, data compression, and measuring the information content of diagnostic tests.

In [ ]:
class HealthcareInformationTheory:
    """
    Information theory applications for healthcare data analysis.
    
    This class implements entropy, mutual information, and other
    information-theoretic measures for clinical data analysis.
    """
    
    def __init__(self):
        """Initialize information theory calculator."""
        pass
    
    def calculate_entropy(self, probabilities):
        """
        Calculate Shannon entropy.
        
        Parameters:
        -----------
        probabilities : array-like
            Probability distribution
            
        Returns:
        --------
        float : Entropy in bits
        """
        probabilities = np.array(probabilities)
        # Remove zero probabilities to avoid log(0)
        probabilities = probabilities[probabilities > 0]
        return -np.sum(probabilities * np.log2(probabilities))
    
    def calculate_conditional_entropy(self, joint_probs):
        """
        Calculate conditional entropy H(Y|X).
        
        Parameters:
        -----------
        joint_probs : 2D array
            Joint probability distribution P(X,Y)
            
        Returns:
        --------
        float : Conditional entropy
        """
        joint_probs = np.array(joint_probs)
        marginal_x = np.sum(joint_probs, axis=1)
        
        conditional_entropy = 0
        for i in range(joint_probs.shape[0]):
            if marginal_x[i] > 0:
                conditional_probs = joint_probs[i, :] / marginal_x[i]
                entropy_y_given_x = self.calculate_entropy(conditional_probs)
                conditional_entropy += marginal_x[i] * entropy_y_given_x
        
        return conditional_entropy
    
    def calculate_mutual_information(self, joint_probs):
        """
        Calculate mutual information I(X;Y).
        
        Parameters:
        -----------
        joint_probs : 2D array
            Joint probability distribution P(X,Y)
            
        Returns:
        --------
        float : Mutual information
        """
        joint_probs = np.array(joint_probs)
        marginal_x = np.sum(joint_probs, axis=1)
        marginal_y = np.sum(joint_probs, axis=0)
        
        # Calculate entropies
        entropy_x = self.calculate_entropy(marginal_x)
        entropy_y = self.calculate_entropy(marginal_y)
        joint_entropy = self.calculate_entropy(joint_probs.flatten())
        
        # Mutual information = H(X) + H(Y) - H(X,Y)
        return entropy_x + entropy_y - joint_entropy
    
    def diagnostic_information_gain(self, disease_prevalence, test_sensitivity, test_specificity):
        """
        Calculate information gain from a diagnostic test.
        
        Parameters:
        -----------
        disease_prevalence : float
            Prior probability of disease
        test_sensitivity : float
            Test sensitivity
        test_specificity : float
            Test specificity
            
        Returns:
        --------
        dict : Information gain analysis
        """
        # Prior entropy (before test)
        prior_probs = [disease_prevalence, 1 - disease_prevalence]
        prior_entropy = self.calculate_entropy(prior_probs)
        
        # Joint probability distribution: P(Disease, Test Result)
        # Rows: Disease (Present, Absent), Columns: Test (Positive, Negative)
        joint_probs = np.array([
            [disease_prevalence * test_sensitivity, disease_prevalence * (1 - test_sensitivity)],
            [(1 - disease_prevalence) * (1 - test_specificity), (1 - disease_prevalence) * test_specificity]
        ])
        
        # Calculate mutual information
        mutual_info = self.calculate_mutual_information(joint_probs)
        
        # Information gain = Mutual information
        information_gain = mutual_info
        
        # Relative information gain
        relative_gain = information_gain / prior_entropy if prior_entropy > 0 else 0
        
        return {
            'prior_entropy': prior_entropy,
            'information_gain': information_gain,
            'relative_gain': relative_gain,
            'joint_distribution': joint_probs
        }
    
    def feature_selection_mutual_information(self, X, y):
        """
        Rank features by mutual information with target variable.
        
        Parameters:
        -----------
        X : DataFrame
            Feature matrix
        y : array-like
            Target variable
            
        Returns:
        --------
        DataFrame : Features ranked by mutual information
        """
        from sklearn.feature_selection import mutual_info_classif
        
        # Calculate mutual information for each feature
        mi_scores = mutual_info_classif(X, y, random_state=42)
        
        # Create results DataFrame
        feature_ranking = pd.DataFrame({
            'feature': X.columns,
            'mutual_information': mi_scores
        }).sort_values('mutual_information', ascending=False)
        
        return feature_ranking
    
    def plot_information_analysis(self, prevalence_range, sensitivity, specificity):
        """
        Plot information gain across different disease prevalences.
        """
        prevalences = np.linspace(prevalence_range[0], prevalence_range[1], 100)
        information_gains = []
        prior_entropies = []
        
        for prev in prevalences:
            analysis = self.diagnostic_information_gain(prev, sensitivity, specificity)
            information_gains.append(analysis['information_gain'])
            prior_entropies.append(analysis['prior_entropy'])
        
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(prevalences, information_gains, 'b-', linewidth=2, label='Information Gain')
        plt.plot(prevalences, prior_entropies, 'r--', linewidth=2, label='Prior Entropy')
        plt.xlabel('Disease Prevalence')
        plt.ylabel('Information (bits)')
        plt.title(f'Information Gain vs Disease Prevalence\n(Sensitivity={sensitivity:.2f}, Specificity={specificity:.2f})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        relative_gains = np.array(information_gains) / np.array(prior_entropies)
        plt.plot(prevalences, relative_gains, 'g-', linewidth=2)
        plt.xlabel('Disease Prevalence')
        plt.ylabel('Relative Information Gain')
        plt.title('Relative Information Gain')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Example: Information theory analysis
info_theory = HealthcareInformationTheory()

# Example: Diagnostic test information analysis
test_analysis = info_theory.diagnostic_information_gain(
    disease_prevalence=0.1,
    test_sensitivity=0.9,
    test_specificity=0.95
)

print("Diagnostic Test Information Analysis:")
print(f"Prior entropy (uncertainty): {test_analysis['prior_entropy']:.3f} bits")
print(f"Information gain from test: {test_analysis['information_gain']:.3f} bits")
print(f"Relative information gain: {test_analysis['relative_gain']:.1%}")

# Visualize information gain across prevalences
info_theory.plot_information_analysis(
    prevalence_range=(0.01, 0.5),
    sensitivity=0.9,
    specificity=0.95
)

## Section 4: Optimization Techniques for Clinical Machine Learning

Optimization is at the heart of machine learning algorithms. We'll explore optimization techniques specifically relevant to healthcare applications.

In [ ]:
class ClinicalOptimization:
    """
    Optimization techniques for clinical machine learning applications.
    
    This class implements various optimization algorithms with healthcare-specific
    considerations such as class imbalance and clinical constraints.
    """
    
    def __init__(self):
        """Initialize optimization framework."""
        self.optimization_history = []
        
    def logistic_regression_gradient_descent(self, X, y, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
        """
        Implement logistic regression using gradient descent.
        
        This demonstrates the optimization process for a common clinical prediction model.
        
        Parameters:
        -----------
        X : array-like
            Feature matrix
        y : array-like
            Binary target variable
        learning_rate : float
            Learning rate for gradient descent
        max_iterations : int
            Maximum number of iterations
        tolerance : float
            Convergence tolerance
            
        Returns:
        --------
        dict : Optimization results including weights and convergence history
        """
        X = np.array(X)
        y = np.array(y)
        
        # Add intercept term
        X_with_intercept = np.column_stack([np.ones(X.shape[0]), X])
        
        # Initialize weights
        weights = np.random.normal(0, 0.01, X_with_intercept.shape[1])
        
        # Track optimization history
        cost_history = []
        weight_history = []
        
        for iteration in range(max_iterations):
            # Forward pass
            z = X_with_intercept @ weights
            predictions = 1 / (1 + np.exp(-z))  # Sigmoid function
            
            # Calculate cost (log-likelihood)
            cost = -np.mean(y * np.log(predictions + 1e-15) + (1 - y) * np.log(1 - predictions + 1e-15))
            cost_history.append(cost)
            weight_history.append(weights.copy())
            
            # Calculate gradients
            gradients = X_with_intercept.T @ (predictions - y) / len(y)
            
            # Update weights
            new_weights = weights - learning_rate * gradients
            
            # Check convergence
            if np.linalg.norm(new_weights - weights) < tolerance:
                print(f"Converged after {iteration + 1} iterations")
                break
            
            weights = new_weights
        
        return {
            'weights': weights,
            'cost_history': cost_history,
            'weight_history': weight_history,
            'final_cost': cost_history[-1],
            'iterations': len(cost_history)
        }
    
    def clinical_loss_function(self, y_true, y_pred, false_negative_cost=5, false_positive_cost=1):
        """
        Custom loss function that accounts for clinical costs.
        
        In healthcare, false negatives (missing a disease) often have
        higher costs than false positives (unnecessary treatment).
        
        Parameters:
        -----------
        y_true : array-like
            True labels
        y_pred : array-like
            Predicted probabilities
        false_negative_cost : float
            Cost of false negative (missing disease)
        false_positive_cost : float
            Cost of false positive (unnecessary treatment)
            
        Returns:
        --------
        float : Weighted loss
        """
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        
        # Calculate weighted cross-entropy loss
        false_negative_loss = -false_negative_cost * y_true * np.log(y_pred + 1e-15)
        false_positive_loss = -false_positive_cost * (1 - y_true) * np.log(1 - y_pred + 1e-15)
        
        return np.mean(false_negative_loss + false_positive_loss)
    
    def optimize_clinical_threshold(self, y_true, y_pred_proba, false_negative_cost=5, false_positive_cost=1):
        """
        Optimize decision threshold based on clinical costs.
        
        Parameters:
        -----------
        y_true : array-like
            True labels
        y_pred_proba : array-like
            Predicted probabilities
        false_negative_cost : float
            Cost of false negative
        false_positive_cost : float
            Cost of false positive
            
        Returns:
        --------
        dict : Optimal threshold and cost analysis
        """
        thresholds = np.linspace(0.01, 0.99, 100)
        costs = []
        
        for threshold in thresholds:
            y_pred = (y_pred_proba >= threshold).astype(int)
            
            # Calculate confusion matrix elements
            tn = np.sum((y_true == 0) & (y_pred == 0))
            fp = np.sum((y_true == 0) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            tp = np.sum((y_true == 1) & (y_pred == 1))
            
            # Calculate total cost
            total_cost = fn * false_negative_cost + fp * false_positive_cost
            costs.append(total_cost)
        
        # Find optimal threshold
        optimal_idx = np.argmin(costs)
        optimal_threshold = thresholds[optimal_idx]
        optimal_cost = costs[optimal_idx]
        
        return {
            'optimal_threshold': optimal_threshold,
            'optimal_cost': optimal_cost,
            'thresholds': thresholds,
            'costs': costs
        }
    
    def plot_optimization_results(self, optimization_results, threshold_results=None):
        """
        Visualize optimization results.
        """
        if threshold_results is not None:
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        else:
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # Plot cost convergence
        axes[0].plot(optimization_results['cost_history'], 'b-', linewidth=2)
        axes[0].set_xlabel('Iteration')
        axes[0].set_ylabel('Cost')
        axes[0].set_title('Cost Function Convergence')
        axes[0].grid(True, alpha=0.3)
        
        # Plot weight evolution
        weight_history = np.array(optimization_results['weight_history'])
        for i in range(weight_history.shape[1]):
            axes[1].plot(weight_history[:, i], label=f'Weight {i}', alpha=0.7)
        axes[1].set_xlabel('Iteration')
        axes[1].set_ylabel('Weight Value')
        axes[1].set_title('Weight Evolution During Training')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # Plot threshold optimization if provided
        if threshold_results is not None:
            axes[2].plot(threshold_results['thresholds'], threshold_results['costs'], 'r-', linewidth=2)
            axes[2].axvline(threshold_results['optimal_threshold'], color='green', linestyle='--', 
                           label=f'Optimal: {threshold_results["optimal_threshold"]:.3f}')
            axes[2].set_xlabel('Decision Threshold')
            axes[2].set_ylabel('Total Clinical Cost')
            axes[2].set_title('Clinical Cost vs Decision Threshold')
            axes[2].legend()
            axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Generate example clinical data for optimization
np.random.seed(42)
n_patients = 1000

# Simulate clinical features
age = np.random.normal(65, 15, n_patients)
bmi = np.random.normal(28, 5, n_patients)
glucose = np.random.normal(120, 30, n_patients)
bp_systolic = np.random.normal(140, 20, n_patients)

# Create feature matrix
X_clinical = np.column_stack([age, bmi, glucose, bp_systolic])

# Generate outcome (e.g., diabetes diagnosis)
# Higher risk with age, BMI, glucose, and blood pressure
risk_score = 0.02 * age + 0.1 * bmi + 0.01 * glucose + 0.005 * bp_systolic - 8
probability = 1 / (1 + np.exp(-risk_score))
y_clinical = np.random.binomial(1, probability, n_patients)

print(f"Clinical dataset: {n_patients} patients, {X_clinical.shape[1]} features")
print(f"Outcome prevalence: {y_clinical.mean():.1%}")

# Perform optimization
optimizer = ClinicalOptimization()

# Train model using gradient descent
optimization_results = optimizer.logistic_regression_gradient_descent(
    X_clinical, y_clinical, learning_rate=0.001, max_iterations=1000
)

print(f"\nOptimization completed in {optimization_results['iterations']} iterations")
print(f"Final cost: {optimization_results['final_cost']:.4f}")
print(f"Final weights: {optimization_results['weights']}")

# Generate predictions for threshold optimization
X_with_intercept = np.column_stack([np.ones(X_clinical.shape[0]), X_clinical])
y_pred_proba = 1 / (1 + np.exp(-(X_with_intercept @ optimization_results['weights'])))

# Optimize clinical threshold
threshold_results = optimizer.optimize_clinical_threshold(
    y_clinical, y_pred_proba, false_negative_cost=5, false_positive_cost=1
)

print(f"\nOptimal clinical threshold: {threshold_results['optimal_threshold']:.3f}")
print(f"Optimal total cost: {threshold_results['optimal_cost']:.0f}")

# Visualize results
optimizer.plot_optimization_results(optimization_results, threshold_results)

## Section 5: Causal Inference for Healthcare Research

Causal inference is crucial for understanding treatment effects and making evidence-based clinical decisions. We'll implement key causal inference methods.

In [ ]:
class CausalInferenceHealthcare:
    """
    Causal inference methods for healthcare research.
    
    This class implements various causal inference techniques including
    propensity score matching, instrumental variables, and difference-in-differences.
    """
    
    def __init__(self):
        """Initialize causal inference framework."""
        pass
    
    def propensity_score_matching(self, X, treatment, outcome, caliper=0.1):
        """
        Perform propensity score matching for causal inference.
        
        Parameters:
        -----------
        X : DataFrame
            Covariate matrix
        treatment : array-like
            Binary treatment indicator
        outcome : array-like
            Outcome variable
        caliper : float
            Maximum distance for matching
            
        Returns:
        --------
        dict : Matching results and treatment effect estimate
        """
        from sklearn.linear_model import LogisticRegression
        from sklearn.neighbors import NearestNeighbors
        
        # Estimate propensity scores
        ps_model = LogisticRegression(random_state=42)
        ps_model.fit(X, treatment)
        propensity_scores = ps_model.predict_proba(X)[:, 1]
        
        # Separate treated and control groups
        treated_idx = np.where(treatment == 1)[0]
        control_idx = np.where(treatment == 0)[0]
        
        # Perform nearest neighbor matching
        nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
        nn.fit(propensity_scores[control_idx].reshape(-1, 1))
        
        matched_pairs = []
        for treated_i in treated_idx:
            distances, indices = nn.kneighbors(
                propensity_scores[treated_i].reshape(1, -1)
            )
            
            if distances[0][0] <= caliper:
                control_match_idx = control_idx[indices[0][0]]
                matched_pairs.append((treated_i, control_match_idx))
        
        # Calculate treatment effect
        if len(matched_pairs) > 0:
            treated_outcomes = [outcome[pair[0]] for pair in matched_pairs]
            control_outcomes = [outcome[pair[1]] for pair in matched_pairs]
            
            ate = np.mean(treated_outcomes) - np.mean(control_outcomes)
            
            # Calculate standard error
            differences = np.array(treated_outcomes) - np.array(control_outcomes)
            se = np.std(differences) / np.sqrt(len(differences))
            
            # 95% confidence interval
            ci_lower = ate - 1.96 * se
            ci_upper = ate + 1.96 * se
        else:
            ate = np.nan
            se = np.nan
            ci_lower = np.nan
            ci_upper = np.nan
        
        return {
            'propensity_scores': propensity_scores,
            'matched_pairs': matched_pairs,
            'n_matched': len(matched_pairs),
            'ate': ate,
            'standard_error': se,
            'confidence_interval': (ci_lower, ci_upper),
            'ps_model': ps_model
        }
    
    def instrumental_variable_analysis(self, X, treatment, outcome, instrument):
        """
        Perform instrumental variable analysis using two-stage least squares.
        
        Parameters:
        -----------
        X : DataFrame
            Covariate matrix
        treatment : array-like
            Treatment variable (can be continuous)
        outcome : array-like
            Outcome variable
        instrument : array-like
            Instrumental variable
            
        Returns:
        --------
        dict : IV analysis results
        """
        from sklearn.linear_model import LinearRegression
        
        # First stage: regress treatment on instrument and covariates
        X_with_instrument = np.column_stack([X, instrument])
        first_stage = LinearRegression()
        first_stage.fit(X_with_instrument, treatment)
        predicted_treatment = first_stage.predict(X_with_instrument)
        
        # Check instrument strength (F-statistic)
        residuals_first = treatment - predicted_treatment
        f_stat = np.var(predicted_treatment) / np.var(residuals_first) * (len(treatment) - X_with_instrument.shape[1] - 1)
        
        # Second stage: regress outcome on predicted treatment and covariates
        X_with_predicted_treatment = np.column_stack([X, predicted_treatment])
        second_stage = LinearRegression()
        second_stage.fit(X_with_predicted_treatment, outcome)
        
        # Treatment effect is the coefficient on predicted treatment
        treatment_effect = second_stage.coef_[-1]
        
        return {
            'treatment_effect': treatment_effect,
            'first_stage_model': first_stage,
            'second_stage_model': second_stage,
            'f_statistic': f_stat,
            'instrument_strength': 'Strong' if f_stat > 10 else 'Weak',
            'predicted_treatment': predicted_treatment
        }
    
    def plot_causal_analysis(self, ps_results, iv_results=None):
        """
        Visualize causal analysis results.
        """
        if iv_results is not None:
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        else:
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            axes = axes.reshape(1, -1)
        
        # Propensity score distribution
        treated_ps = ps_results['propensity_scores'][treatment == 1]
        control_ps = ps_results['propensity_scores'][treatment == 0]
        
        axes[0, 0].hist(control_ps, bins=30, alpha=0.7, label='Control', color='blue')
        axes[0, 0].hist(treated_ps, bins=30, alpha=0.7, label='Treated', color='red')
        axes[0, 0].set_xlabel('Propensity Score')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].set_title('Propensity Score Distribution')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Treatment effect visualization
        ate = ps_results['ate']
        ci_lower, ci_upper = ps_results['confidence_interval']
        
        axes[0, 1].errorbar([0], [ate], yerr=[[ate - ci_lower], [ci_upper - ate]], 
                           fmt='o', capsize=10, capthick=2, markersize=8, color='red')
        axes[0, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        axes[0, 1].set_xlim(-0.5, 0.5)
        axes[0, 1].set_ylabel('Treatment Effect')
        axes[0, 1].set_title(f'Average Treatment Effect\n{ate:.3f} [{ci_lower:.3f}, {ci_upper:.3f}]')
        axes[0, 1].set_xticks([])
        axes[0, 1].grid(True, alpha=0.3)
        
        # IV analysis plots if provided
        if iv_results is not None:
            # First stage relationship
            axes[1, 0].scatter(instrument, treatment, alpha=0.6)
            axes[1, 0].plot(instrument, iv_results['predicted_treatment'], 'r-', linewidth=2)
            axes[1, 0].set_xlabel('Instrument')
            axes[1, 0].set_ylabel('Treatment')
            axes[1, 0].set_title(f'First Stage (F-stat: {iv_results["f_statistic"]:.1f})')
            axes[1, 0].grid(True, alpha=0.3)
            
            # IV treatment effect
            iv_effect = iv_results['treatment_effect']
            axes[1, 1].bar(['PS Matching', 'IV Estimate'], [ate, iv_effect], 
                          color=['blue', 'green'], alpha=0.7)
            axes[1, 1].set_ylabel('Treatment Effect')
            axes[1, 1].set_title('Comparison of Causal Estimates')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Generate example data for causal inference
np.random.seed(42)
n_patients = 2000

# Confounders (age, comorbidity score, severity)
age = np.random.normal(65, 15, n_patients)
comorbidity_score = np.random.poisson(2, n_patients)
severity = np.random.normal(0, 1, n_patients)

# Instrumental variable (e.g., physician preference, distance to specialist)
instrument = np.random.normal(0, 1, n_patients)

# Treatment assignment (influenced by confounders and instrument)
treatment_logit = (
    0.02 * age + 
    0.3 * comorbidity_score + 
    0.4 * severity + 
    0.5 * instrument +  # Instrument effect
    np.random.normal(0, 1, n_patients)
)
treatment_prob = 1 / (1 + np.exp(-treatment_logit))
treatment = np.random.binomial(1, treatment_prob, n_patients)

# Outcome (influenced by confounders and treatment)
true_treatment_effect = 2.0  # True causal effect
outcome = (
    0.1 * age + 
    0.5 * comorbidity_score + 
    1.0 * severity + 
    true_treatment_effect * treatment +  # True causal effect
    np.random.normal(0, 2, n_patients)
)

# Create covariate matrix
X_causal = pd.DataFrame({
    'age': age,
    'comorbidity_score': comorbidity_score,
    'severity': severity
})

print(f"Causal inference dataset: {n_patients} patients")
print(f"Treatment rate: {treatment.mean():.1%}")
print(f"True treatment effect: {true_treatment_effect}")

# Perform causal analysis
causal_analyzer = CausalInferenceHealthcare()

# Propensity score matching
ps_results = causal_analyzer.propensity_score_matching(
    X_causal, treatment, outcome, caliper=0.1
)

print(f"\nPropensity Score Matching Results:")
print(f"Matched pairs: {ps_results['n_matched']} out of {np.sum(treatment)} treated patients")
print(f"Estimated ATE: {ps_results['ate']:.3f} (SE: {ps_results['standard_error']:.3f})")
print(f"95% CI: [{ps_results['confidence_interval'][0]:.3f}, {ps_results['confidence_interval'][1]:.3f}]")

# Instrumental variable analysis
iv_results = causal_analyzer.instrumental_variable_analysis(
    X_causal, treatment, outcome, instrument
)

print(f"\nInstrumental Variable Analysis Results:")
print(f"Estimated treatment effect: {iv_results['treatment_effect']:.3f}")
print(f"First-stage F-statistic: {iv_results['f_statistic']:.1f} ({iv_results['instrument_strength']} instrument)")

# Visualize results
causal_analyzer.plot_causal_analysis(ps_results, iv_results)

## Summary and Next Steps

In this chapter, we've covered the essential mathematical foundations for healthcare AI:

1. **Probability Theory**: Diagnostic test interpretation and clinical decision making
2. **Bayesian Inference**: Updating beliefs with new evidence
3. **Information Theory**: Measuring information content and feature selection
4. **Optimization**: Training clinical prediction models
5. **Causal Inference**: Estimating treatment effects from observational data

### Key Takeaways:

- **Clinical Context Matters**: Mathematical methods must be adapted for healthcare-specific considerations like class imbalance, clinical costs, and regulatory requirements
- **Uncertainty Quantification**: Healthcare decisions require proper uncertainty estimation and confidence intervals
- **Causal Reasoning**: Correlation is not causation - proper causal inference methods are essential for treatment effect estimation
- **Interpretability**: Mathematical models in healthcare must be interpretable and explainable to clinicians

### Next Chapter Preview:

Chapter 3 will build on these mathematical foundations to cover **Healthcare Data Engineering**, including:
- FHIR and HL7 standards implementation
- Real-time data processing pipelines
- Data quality and validation frameworks
- Privacy-preserving data processing

### Exercises:

1. Modify the diagnostic test calculator to handle multiple tests in sequence
2. Implement a Bayesian A/B test framework for clinical trials
3. Create a feature selection pipeline using mutual information
4. Develop a cost-sensitive optimization algorithm for imbalanced clinical data
5. Implement difference-in-differences analysis for policy evaluation